In [3]:
import xarray as xr
import sys
sys.path.append('/home/563/rl5183/PhD-research/Functions')
import functions as func

In [4]:
# Read in model ts data 
multi_model = func.read_in_cmip_models('/g/data/eg3/mf3225/CMIP_TS/CMIP6/historical', 'r1i1p1f1', 'ts', '1850-01', '2015-01')
# Order models alphabetically 
multi_model = multi_model.sortby('model')
# Select out surface temperature data 
data = multi_model.ts

In [5]:
# Select out the 20 models we are using 
keep = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CanESM5', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-g3',
        'GFDL-CM4', 'GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0', 'KACE-1-0-G', 'KIOST-ESM', 'MIROC6', 'MPI-ESM1-2-HR',
        'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3', 'NorESM2-LM', 'NorESM2-MM']

data = data.sel(model=keep)

In [6]:
# Quadratically detrend temp data 
data_dt = xr.apply_ufunc(func.quad_detrend, data, 
                             input_core_dims=[['time']],
                             output_core_dims=[['time']],
                             vectorize=True,
                             dask='parallelized')

data_dt = data_dt.transpose(*data.dims).chunk('auto').sel(time=slice('1900-01', '2014-12'))

In [7]:
data_dt.to_netcdf('/g/data/w40/rl5183/CMIP6_20_ts_quad_detrend_1900_2015.nc')